<a href="https://colab.research.google.com/github/casangi/cngi_prototype/blob/master/docs/calibration/self_calibration_solve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In Development

(https://casaguides.nrao.edu/index.php/First_Look_at_Imaging) will be used. 

This walkthrough is designed to be run in a Jupyter notebook on Google Colaboratory. To open the notebook in colab, go [here](https://colab.research.google.com/github/casangi/cngi_prototype/blob/master/docs/calibration/self_calibration_solve.ipynb).

## Installation and Dataset Download


In [2]:
import os
#os.system("pip install --extra-index-url https://test.pypi.org/simple/ cngi-prototype==0.0.70")

#sis14_twhya_field_5_lsrk.vis.zarr https://drive.google.com/file/d/1QRHqrYN_9jhGx0s0URzTYUVu5JsjJCEl/view?usp=sharing
id = '1QRHqrYN_9jhGx0s0URzTYUVu5JsjJCEl'
os.system('curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=%s"' % id)
os.system('curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk \'/download/ {print $NF}\' ./cookie`&id=%s" -o vis.zarr.zip' % id)
os.system('unzip vis.zarr.zip')



print('complete') 

complete


## Dataset

How to recreate data:

https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated_flagged.ms.tar

```python
mstransform('sis14_twhya_calibrated_flagged.ms',
             outputvis='sis14_twhya_field_5_lsrk.ms',
             regridms=True, outframe='LSRK', datacolumn='corrected', field='5')

tclean(vis='sis14_twhya_field_5_lsrk.ms/',
       imagename='first_image',
       spw='',
       specmode='mfs',
       deconvolver='hogbom',
       nterms=1,
       gridder='standard',
       imsize=[250,250],
       cell=['0.1arcsec'],
       weighting='natural',
       threshold='0mJy',
       niter=5000,
       savemodel='modelcolumn')

from cngi.conversion import convert_ms
vis_mxds = convert_ms('sis14_twhya_field_5_lsrk.ms/',chunks=(27,21,32,2))
```

In [10]:
#Reload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import xarray as xr
from cngi.dio import read_vis, read_image
from ngcasa.calibration import solve_calibration

xr.set_options(display_style="html")
vis_mxds = read_vis("sis14_twhya_field_5_lsrk.vis.zarr",chunks={'time':270,'baseline':210,'chan':192,'pol':2})

print(vis_mxds.xds0)

cal_xds = None
sel_parms = {}
sel_parms['xds'] = 'xds0'
sel_parms['data_group_in'] = {'data':'DATA','model_data':'MODEL_DATA','weight':'WEIGHT'}

solve_parms= {}

cal_solution = solve_calibration(vis_mxds, cal_xds, solve_parms, sel_parms)


cal_solution.compute()


<xarray.Dataset>
Dimensions:         (baseline: 210, chan: 384, pol: 2, pol_id: 1, spw_id: 1, time: 270, uvw_index: 3)
Coordinates:
  * baseline        (baseline) int64 0 1 2 3 4 5 6 ... 204 205 206 207 208 209
  * chan            (chan) float64 3.725e+11 3.725e+11 ... 3.728e+11 3.728e+11
    chan_width      (chan) float64 dask.array<chunksize=(192,), meta=np.ndarray>
    effective_bw    (chan) float64 dask.array<chunksize=(192,), meta=np.ndarray>
  * pol             (pol) int32 9 12
  * pol_id          (pol_id) int32 0
    resolution      (chan) float64 dask.array<chunksize=(192,), meta=np.ndarray>
  * spw_id          (spw_id) int32 0
  * time            (time) datetime64[ns] 2012-11-19T07:56:26.544000626 ... 2...
Dimensions without coordinates: uvw_index
Data variables:
    ANTENNA1        (time, baseline) int32 dask.array<chunksize=(270, 210), meta=np.ndarray>
    ANTENNA2        (time, baseline) int32 dask.array<chunksize=(270, 210), meta=np.ndarray>
    ARRAY_ID        (time, base

array([[[[ 2.23877931e+00+2.07556558e+00j,
          -2.38666981e-01+1.75645828e+00j],
         [ 2.08931971e+00-8.03868294e+00j,
          -1.06614399e+01-2.51691127e+00j],
         [ 1.47529325e+01-1.34241610e+01j,
           3.06206512e+00-8.28583622e+00j],
         ...,
         [-1.69494038e+01+4.37973976e-01j,
          -9.17492771e+00-1.17050922e+00j],
         [-9.40954685e+00-1.60929146e+01j,
           1.00802457e+00+8.60961342e+00j],
         [ 8.43718529e+00+8.87509727e+00j,
          -5.89503622e+00-1.00731623e+00j]],

        [[-7.03216028e+00-7.43496990e+00j,
           6.96008444e+00-6.43356514e+00j],
         [ 5.51236057e+00+9.03958607e+00j,
           6.15174246e+00+1.15197649e+01j],
         [ 1.40511160e+01+1.40959272e+01j,
           7.58338690e+00-4.24955511e+00j],
         ...,
         [-5.23491526e+00+5.86703682e+00j,
          -1.21352243e+01+5.36098599e-01j],
         [ 6.06391907e+00-1.34663248e+01j,
          -2.38290024e+00-9.25668621e+00j],
         [-7.